# Dead simple version

In [49]:
import doctest
import numpy as np
from scipy.spatial.distance import jaccard


def jaccard_index(labels, preds, num_classes=3):
    """
    >>> jaccard_index(np.array([0,0,1,1,2,2]), np.array([0,0,1,1,2,2]))
    [1.0, 1.0, 1.0]
    
    >>> jaccard_index(np.array([0,0,1,1,2,2]), np.array([2,2,0,0,1,1]))
    [0.0, 0.0, 0.0]
    
    >>> jaccard_index(np.array([0,0,1,1,2,2]), np.array([0,0,1,0,1,1]))
    [1.0, 0.5, 0.0]
    
    >>> jaccard_index(np.array([[0,0,1,1,2,2], [0,0,1,1,2,2]]), np.array([[0,0,1,0,1,1], [0,0,1,1,2,2]]))
    [1.0, 0.75, 0.5]
    """
    
    result = []
    
    for i in range(num_classes):
        mask = labels == i
    
        inter = (labels[mask] == preds[mask]).astype(np.int).sum()
        union = len(labels[mask])
    
        if union == 0:
            result.append(0)
        else:
            result.append(float(inter)/union)    
    
    return result


doctest.testmod()

TestResults(failed=0, attempted=4)

# Confusion Matrix version

In [39]:
class ConfusionMatrix(object):
    def __init__(self, num_classes):
        self.num_classes = num_classes
        self.matrix = np.zeros((num_classes, num_classes))
        
    def update(self, actual, predicted):
        actual = actual.ravel()
        predicted = predicted.ravel()
        
        for a, p in zip(actual, predicted):
            self.matrix[p, a] += 1
            
    def get_matrix(self):
        return self.matrix
    
    def get_iou(self):
        return np.diag(self.matrix) / self.matrix.sum(axis=0)

In [47]:
a, b = np.array([0,0,1,1,2,2]), np.array([0,0,1,1,2,2])
cm = ConfusionMatrix(3)
cm.update(a, b)
assert (cm.get_iou() == np.array([1.0, 1.0, 1.0])).all()

a, b = np.array([0,0,1,1,2,2]), np.array([2,2,0,0,1,1])
cm = ConfusionMatrix(3)
cm.update(a, b)
assert (cm.get_iou() == np.array([0.0, 0.0, 0.0])).all()

a, b = np.array([[0,0,1,1,2,2], [0,0,1,1,2,2]]), np.array([[0,0,1,0,1,1], [0,0,1,1,2,2]])
cm = ConfusionMatrix(3)
cm.update(a, b)
assert (cm.get_iou() == np.array([1.0, 0.75, 0.5])).all()

# PyTorch version

In [137]:
import torch

class ConfusionMatrix(object):
    def __init__(self, num_classes):
        self.num_classes = num_classes
        self.matrix = torch.zeros((num_classes, num_classes))
        
    def update(self, actual, predicted):
        s = actual.flatten()*self.num_classes + predicted.flatten()
        self.matrix = self.matrix.put_(s, torch.ones_like(s).float(), accumulate=True).T
    
    def get_matrix(self):
        return self.matrix
    
    def get_iou(self):
        return torch.diag(self.matrix) / self.matrix.sum(dim=0)
    
    def get_miou(self):
        return torch.mean(self.get_iou())

In [140]:
a, b = torch.tensor([0,0,1,1,2,2]), torch.tensor([0,0,1,1,2,2])
cm = ConfusionMatrix(3)
cm.update(a, b)
assert torch.all(cm.get_iou() == torch.tensor([1.0, 1.0, 1.0]))

a, b = torch.tensor([0,0,1,1,2,2]), torch.tensor([2,2,0,0,1,1])
cm = ConfusionMatrix(3)
cm.update(a, b)
assert torch.all(cm.get_iou() == torch.tensor([0.0, 0.0, 0.0]))

a, b = torch.tensor([[0,0,1,1,2,2], [0,0,1,1,2,2]]), torch.tensor([[0,0,1,0,1,1], [0,0,1,1,2,2]])
cm = ConfusionMatrix(3)
cm.update(a, b)
assert torch.all(cm.get_iou() == torch.tensor([1.0, 0.75, 0.5]))

a, b = torch.tensor([[0,0,1,1,2,2,3,3], [0,0,1,1,2,2,3,3]]), torch.tensor([[0,0,1,0,1,1,2,2], [0,0,1,1,2,2,3,3]])
cm = ConfusionMatrix(4)
cm.update(a, b)
assert torch.all(cm.get_iou() == torch.tensor([1.0, 0.75, 0.5,0.5]))
assert cm.get_miou() == torch.mean(torch.tensor([1.0, 0.75, 0.5,0.5]))

AssertionError: 